Preprocessing - **MinMaxScaling**  
Algorithm - **OneClassSVM**  
Feature - **Real + Imaginary + Magnitude**

In [97]:
# ALPLA demo- Implementation with a small scale dataset

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)


# Load dataset
# measurement = np.load('../test/dataset/meas_symm_1.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_2.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_3.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_4.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_5.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_varspeed_1.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_asymm_1.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_asymm_2.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_asymm_reflector_1.npz', allow_pickle=False)
measurement = np.load('../test/dataset/meas_asymm_reflector_2.npz', allow_pickle=False)

header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]

# Train-test split
# trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)

trainCIR, tempCIR = train_test_split(data_cir, test_size=0.4, random_state=42)
testCIR, evalCIR = train_test_split(tempCIR, test_size=0.5, random_state=42)

print(f'trainData - {trainCIR.shape}')
print(f'testData - {testCIR.shape}')

# Define channels
alice_channel = 3  # A -> B (legitimate)
eve_channel = 6  # E -> B (illegitimate)

initial_cirs = 100
window_size = 200  # Define a fixed window size


trainData - (600, 15, 251, 2)
testData - (200, 15, 251, 2)


In [98]:
def cirto64Samples(cirs):
    print(f'Input data shape: {cirs.shape}')
    real = cirs[:, :, 0]
    imag = cirs[:, :, 1]
    
    # Number of signals
    num_signals = real.shape[0]  # 3 in this case
    
    # Initialize lists to store the focused samples
    imp_real_parts = []
    imp_imag_parts = []
    img_mag_parts = []
    
    for i in range(num_signals):
        # Calculate the magnitude
        magnitude = np.abs(real[i] + 1j * imag[i])
        
        # find the peak index
        peak_index = np.argmax(magnitude)
        
        # Calculate the start and end indices for the focused part
        start_index = max(0, peak_index - 32)
        end_index = min(magnitude.shape[0], peak_index + 32)
        
        # Extract the part of the signal around the peak
        real_part_focus = real[i, start_index:end_index]
        imag_part_focus = imag[i, start_index:end_index]
        mag_part_focus = magnitude[start_index:end_index]
        
        imp_real_parts.append(real_part_focus)
        imp_imag_parts.append(imag_part_focus)
        img_mag_parts.append(mag_part_focus)
        

    # Convert lists back to arrays for further processing if needed
    imp_real_parts = np.array(imp_real_parts)
    imp_imag_parts = np.array(imp_imag_parts)
    img_mag_parts = np.array(img_mag_parts)

    return imp_real_parts, imp_imag_parts, img_mag_parts


def getRealImagMag(cirs):
    real = cirs[:, :, 0]
    imag = cirs[:, :, 1]
    mag = np.abs(real + 1j * imag)
    return real, imag, mag

In [99]:
def update_features(features, new_cir, window_size):
    # print(f'features shape: {features.shape}')
    if features.shape[0] >= window_size:
        # Remove the oldest CIR (first row)
        features = np.delete(features, 0, axis=0)
    
    # Append the new CIR to the end 
    updated_features = np.vstack([features, new_cir])
    return updated_features

In [100]:
#------------------------------------------- Training  ------------------------------------------------
scaler = MinMaxScaler()

# Initialize OCC-SVM model
ocsvm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.1)


# Training data --------
# Feature Extraction
train_alice_cirs = trainCIR[:initial_cirs, alice_channel, :, :]
train_alice_real, train_alice_imag, train_alice_mag = getRealImagMag(train_alice_cirs)
train_alice_features = np.hstack((train_alice_real, train_alice_imag, train_alice_mag))
train_features = train_alice_features

# Scaling
train_features_scaled = scaler.fit_transform(train_features)
print(f'train_features_scaled - {train_features_scaled.shape}')

# Training the model
ocsvm.fit(train_features_scaled)



train_features_scaled - (100, 753)


OneClassSVM(gamma='auto', nu=0.1)

In [101]:
# --------------------------------------------------- Testing Part ---------------------------------------------------
# Testing Data --------
test_alice_cirs = testCIR[:, alice_channel, :, :]
test_alice_real, test_alice_imag, test_alice_mag = getRealImagMag(test_alice_cirs)
test_alice_features = np.hstack((test_alice_real, test_alice_imag, test_alice_mag))

test_eve_cirs = testCIR[:, eve_channel, :, :]
test_eve_real, test_eve_imag, test_eve_mag = getRealImagMag(test_eve_cirs)
test_eve_features = np.hstack((test_eve_real, test_eve_imag, test_eve_mag))

test_features = np.vstack((test_alice_features, test_eve_features))
# Scaling
test_features_scaled = scaler.transform(test_features)

# labels
test_alice_label = np.ones(test_alice_cirs.shape[0])  # '1' for Alice
test_eve_labels = np.full(test_eve_cirs.shape[0], -1)       # '-1' for Eve
test_labels = np.hstack((test_alice_label, test_eve_labels))  # (3200,)
print(f'test_labels - {test_labels.shape}')

print(f'test_features_scaled - {test_features_scaled.shape[0]}')
predictions = []
# print(test_features_scaled[0])
for index, cir in enumerate(test_features_scaled):
    cir_reshaped = cir.reshape(1, -1) # (1, 753)
    prediction = ocsvm.predict(cir_reshaped)
    predictions.append(prediction[0])
    
    if prediction == 1:
        train_features_scaled = update_features(train_features_scaled, cir_reshaped, window_size)
        ocsvm.fit(train_features_scaled)

predictions = np.array(predictions)
print(train_features_scaled.shape)
# print(f'Predictions: {predictions}')



accuracy = np.mean(predictions == test_labels)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")


tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[-1, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# # Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# # False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# # Gamma calculation
gamma = (tp + fn) / (tn + fp)

# # Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")

test_labels - (400,)
test_features_scaled - 400


(200, 753)
Classification Accuracy: 67.50%
tp: 140
tn: 130
fp: 70
fn: 60
MDR: 0.35
FAR: 0.3
AR: 0.675


In [102]:
# ---------------------------------------------- Evaluation Part ------------------------------------------------
# Evaluation Data - Feature Extraction
eval_alice_cirs = evalCIR[:, alice_channel, :, :]
eval_alice_real, eval_alice_imag, eval_alice_mag = getRealImagMag(eval_alice_cirs)
eval_alice_features = np.hstack((eval_alice_real, eval_alice_imag, eval_alice_mag))

eval_eve_cirs = evalCIR[:, eve_channel, :, :]
eval_eve_real, eval_eve_imag, eval_eve_mag = getRealImagMag(eval_eve_cirs)
eval_eve_features = np.hstack((eval_eve_real, eval_eve_imag, eval_eve_mag))

# Stack evaluation features
eval_features = np.vstack((eval_alice_features, eval_eve_features))

eval_features_scaled = scaler.transform(eval_features)

# Labels for evaluation data
eval_alice_labels = np.ones(eval_alice_cirs.shape[0])  # Label '1' for Alice
eval_eve_labels = np.full(eval_eve_cirs.shape[0], -1)  # Label '-1' for Eve
eval_labels = np.hstack((eval_alice_labels, eval_eve_labels))

# Make predictions on evaluation data
eval_predictions = []
for cir in eval_features_scaled:
    cir_reshaped = cir.reshape(1, -1)
    prediction = ocsvm.predict(cir_reshaped)
    eval_predictions.append(prediction[0])

eval_predictions = np.array(eval_predictions)

# ---------------------------------------------- Evaluation ------------------------------------------------
eval_accuracy  = np.mean(eval_predictions == eval_labels)
print(f"Classification Accuracy: {eval_accuracy * 100:.2f}%")


tn, fp, fn, tp = confusion_matrix(eval_labels, eval_predictions, labels=[-1, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# # Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# # False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# # Gamma calculation
gamma = (tp + fn) / (tn + fp)

# # Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Classification Accuracy: 61.25%
tp: 116
tn: 129
fp: 71
fn: 84
MDR: 0.355
FAR: 0.42
AR: 0.6125


In [103]:
# # ------------------ Training the model ------------------
# alice_real = X_train[:50, 3, :, 0]
# alice_imag = X_train[:50, 3, :, 1]
# alice_mag = np.abs(alice_real + 1j * alice_imag)

# # alice_real, alice_imag, alice_mag = servesImportant64Samples(alice_real_251, alice_imag_251)

# # feature set
# combined_train_features = np.column_stack((alice_real, alice_imag, alice_mag))

# # Scaling
# scaler.fit(combined_train_features)
# scaled_train_features = scaler.transform(combined_train_features)

# # Applying PCA
# pca.fit(scaled_train_features)
# reduced_train_features = pca.transform(combined_train_features)

# # Training
# ocsvm_new.fit(reduced_train_features)

# # Save the vectors
# alice_features = reduced_train_features

In [104]:
# # ------------------ Testing the model ------------------
# true_labels = []
# predictions = []

# # Buffer to hold positive samples before retraining
# positive_sample_buffer = []
# batch_size = 800  # Retrain after collecting 800 new positive samples

# # Channels to test
# selected_channel = [3, 6]


# for cir in range(X_test.shape[0]):
#     for channel in selected_channel:
        
#         # Setting true label
#         if channel == 3:
#             true_label = 1  # Legitimate user (Alice)
#         else:
#             true_label = -1  # Illegitimate user (Not Alice)
        
#         true_labels.append(true_label)
        
        
#         incoming_real = X_test[cir, channel, :, 0].reshape(1, -1)
#         incoming_imag = X_test[cir, channel, :, 1].reshape(1, -1)
#         incoming_mag = np.abs(incoming_real + 1j * incoming_imag).reshape(1, -1)

#         combine_test_features = np.column_stack((incoming_real, incoming_imag, incoming_mag))
        
#         combine_test_features_scaled = scaler.transform(combine_test_features)
        
#         reduced_test_features = pca.transform(combine_test_features_scaled)
        
#         prediction = ocsvm_new.predict(reduced_test_features)
        
#         predictions.append(prediction[0])
        
#         # Add new positive sample to buffer if prediction is 1 (legitimate user)
#         if prediction[0] == 1:
#             positive_sample_buffer.append(reduced_test_features)

#             # Retrain if buffer has enough positive samples
#             if len(positive_sample_buffer) >= batch_size:
#                 alice_features = update_features(alice_features, np.vstack(positive_sample_buffer), 500, 100)
                
#                 # Clear the buffer after retraining
#                 positive_sample_buffer.clear()
